In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LeakyReLU
from tensorflow.keras.layers import Input, Reshape, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.layers import Concatenate, Add, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from copy import deepcopy
from sklearn.metrics import classification_report, confusion_matrix
import time



2025-04-25 02:33:48.983377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745548429.178973      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745548429.234560      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Set paths for the dataset
train_dir = '/kaggle/input/malaria-detection-dataset/Dataset/Train'
test_dir = '/kaggle/input/malaria-detection-dataset/Dataset/Test'
parasite_train_dir = os.path.join(train_dir, 'Parasite')
uninfected_train_dir = os.path.join(train_dir, 'Uninfected')
parasite_test_dir = os.path.join(test_dir, 'Parasite')
uninfected_test_dir = os.path.join(test_dir, 'Uninfected')

# Configuration parameters
class Config:
    # Image parameters
    img_width, img_height = 64, 64
    channels = 3
    
    # GAN parameters
    latent_dim = 100
    batch_size = 32
    
    # NAS parameters
    population_size = 10
    generations = 5
    mutation_rate = 0.2
    tournament_size = 3
    
    # Training parameters
    nas_epochs = 3  # Short training for each candidate architecture
    final_epochs = 10  # Longer training for the best architecture
    standard_gan_epochs = 10  # Training for standard GAN
    
    # Operation pool
    operations = [
        'conv3x3', 'conv5x5', 'conv7x7',
        'sep_conv3x3', 'sep_conv5x5',
        'dil_conv3x3', 'dil_conv5x5',
        'skip_connect', 'none'
    ]
    
    # Cell structure
    cell_nodes = 4  # Number of nodes in a cell
    max_edges = 2   # Maximum incoming edges per node



In [3]:
# Data loading and preprocessing functions
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(Config.img_width, Config.img_height),
        batch_size=Config.batch_size,
        class_mode='binary',
        color_mode='rgb'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(Config.img_width, Config.img_height),
        batch_size=Config.batch_size,
        class_mode='binary',
        color_mode='rgb',
        shuffle=False
    )
    
    return train_generator, test_generator

def load_data_from_generator(generator):
    """Load data from a generator and scale to [-1, 1] for tanh activation."""
    generator.reset()
    X_data = []
    y_data = []
    
    # Get all data from generator
    for _ in range(int(np.ceil(generator.samples / Config.batch_size))):
        X, y = next(generator)
        X_data.extend(X)
        y_data.extend(y)
        if len(X_data) >= generator.samples:
            break
    
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    
    # Scale data from [0, 1] to [-1, 1] for tanh activation
    X_data = X_data * 2 - 1
    
    return X_data, y_data




In [4]:
# Define building blocks for the NAS search space
class NASOperations:
    @staticmethod
    def conv_op(x, filters, kernel_size, name):
        return Conv2D(filters, kernel_size, padding='same', name=f"{name}_conv{kernel_size}")(x)
    
    @staticmethod
    def sep_conv_op(x, filters, kernel_size, name):
        return tf.keras.layers.SeparableConv2D(
            filters, kernel_size, padding='same', name=f"{name}_sep_conv{kernel_size}"
        )(x)
    
    @staticmethod
    def dil_conv_op(x, filters, kernel_size, name):
        return Conv2D(
            filters, kernel_size, padding='same', dilation_rate=2, 
            name=f"{name}_dil_conv{kernel_size}"
        )(x)
    
    @staticmethod
    def skip_connect(x, filters, name):
        if x.shape[-1] != filters:
            return Conv2D(filters, 1, padding='same', name=f"{name}_skip_proj")(x)
        return x
    
    @staticmethod
    def apply_operation(x, op_name, filters, node_id):
        name = f"node{node_id}_{op_name}"
        
        if op_name == 'conv3x3':
            return NASOperations.conv_op(x, filters, 3, name)
        elif op_name == 'conv5x5':
            return NASOperations.conv_op(x, filters, 5, name)
        elif op_name == 'conv7x7':
            return NASOperations.conv_op(x, filters, 7, name)
        elif op_name == 'sep_conv3x3':
            return NASOperations.sep_conv_op(x, filters, 3, name)
        elif op_name == 'sep_conv5x5':
            return NASOperations.sep_conv_op(x, filters, 5, name)
        elif op_name == 'dil_conv3x3':
            return NASOperations.dil_conv_op(x, filters, 3, name)
        elif op_name == 'dil_conv5x5':
            return NASOperations.dil_conv_op(x, filters, 5, name)
        elif op_name == 'skip_connect':
            return NASOperations.skip_connect(x, filters, name)
        elif op_name == 'none':
            return None
        else:
            raise ValueError(f"Unknown operation: {op_name}")


In [5]:
# Cell genotype definition
class CellGenotype:
    def __init__(self, edges=None, ops=None):
        # Initialize with random architecture if not provided
        if edges is None or ops is None:
            self.edges = []
            self.ops = []
            
            # For each node in the cell (excluding input nodes)
            for node_id in range(2, 2 + Config.cell_nodes):
                node_edges = []
                node_ops = []
                
                # Randomly select incoming edges (from previous nodes)
                num_edges = random.randint(1, min(node_id, Config.max_edges))
                possible_inputs = list(range(0, node_id))
                selected_inputs = random.sample(possible_inputs, num_edges)
                
                for input_id in selected_inputs:
                    node_edges.append(input_id)
                    # Randomly select an operation (excluding 'none' for initial random sampling)
                    op = random.choice(Config.operations[:-1])
                    node_ops.append(op)
                
                self.edges.append(node_edges)
                self.ops.append(node_ops)
        else:
            self.edges = edges
            self.ops = ops
    
    def mutate(self):
        """Mutate the cell architecture."""
        mutated = deepcopy(self)
        
        # Randomly choose what to mutate
        mutation_type = random.choice(['edge', 'op'])
        
        if mutation_type == 'edge':
            # Choose a random node to mutate its edges
            node_idx = random.randint(0, len(mutated.edges) - 1)
            
            # If we have more than 1 edge, we might remove one
            if len(mutated.edges[node_idx]) > 1 and random.random() < 0.5:
                edge_to_remove = random.randint(0, len(mutated.edges[node_idx]) - 1)
                mutated.edges[node_idx].pop(edge_to_remove)
                mutated.ops[node_idx].pop(edge_to_remove)
            # Otherwise, add a new edge if possible
            elif len(mutated.edges[node_idx]) < min(2 + node_idx, Config.max_edges):
                # Find possible input nodes that are not already connected
                possible_inputs = [i for i in range(0, 2 + node_idx) 
                                  if i not in mutated.edges[node_idx]]
                
                if possible_inputs:
                    new_input = random.choice(possible_inputs)
                    mutated.edges[node_idx].append(new_input)
                    # Choose a random operation
                    new_op = random.choice(Config.operations[:-1])
                    mutated.ops[node_idx].append(new_op)
        
        else:  # Mutate operation
            # Choose a random node
            node_idx = random.randint(0, len(mutated.ops) - 1)
            
            if mutated.ops[node_idx]:  # Ensure there are operations to mutate
                # Choose a random operation in that node
                op_idx = random.randint(0, len(mutated.ops[node_idx]) - 1)
                
                # Replace with a different operation
                current_op = mutated.ops[node_idx][op_idx]
                available_ops = [op for op in Config.operations[:-1] if op != current_op]
                mutated.ops[node_idx][op_idx] = random.choice(available_ops)
                
        return mutated
    
    def crossover(self, other):
        """Perform crossover with another cell genotype."""
        # Ensure both parents have the same structure
        if len(self.edges) != len(other.edges):
            raise ValueError("Parents must have the same number of nodes")
        
        child_edges = []
        child_ops = []
        
        # For each node, randomly choose edges and ops from either parent
        for i in range(len(self.edges)):
            if random.random() < 0.5:
                child_edges.append(deepcopy(self.edges[i]))
                child_ops.append(deepcopy(self.ops[i]))
            else:
                child_edges.append(deepcopy(other.edges[i]))
                child_ops.append(deepcopy(other.ops[i]))
        
        return CellGenotype(edges=child_edges, ops=child_ops)
    
    def to_string(self):
        """Convert genotype to string representation."""
        result = []
        for node_idx, (node_edges, node_ops) in enumerate(zip(self.edges, self.ops)):
            node_str = f"Node {node_idx+2}: "
            for edge, op in zip(node_edges, node_ops):
                node_str += f"({edge}->{node_idx+2}, {op}) "
            result.append(node_str)
        return "\n".join(result)
    
    def __str__(self):
        return self.to_string()




In [6]:
# Generator with NAS architecture
class NASGANGenerator:
    def __init__(self, genotype=None):
        self.genotype = genotype if genotype else CellGenotype()
    
    def build_model(self):
        """Build generator model based on genotype."""
        # Input noise
        noise_input = Input(shape=(Config.latent_dim,))
        
        # Initial dense and reshape
        x = Dense(8*8*256, activation="relu")(noise_input)
        x = Reshape((8, 8, 256))(x)
        
        # Apply cells at different resolutions
        filter_sizes = [256, 128, 64, 32]
        
        for i, filters in enumerate(filter_sizes):
            # Apply NAS cell
            x = self._build_cell(x, filters, f"cell_{i}")
            
            # Upsampling (except for the last level)
            if i < len(filter_sizes) - 1:
                x = UpSampling2D()(x)
        
        # Final output layer
        out = Conv2D(Config.channels, kernel_size=3, padding="same", activation="tanh")(x)
        
        # Create and return model
        return Model(noise_input, out)
    
    def _build_cell(self, x, filters, name_prefix):
        """Build a single cell based on genotype."""
        nodes = [None] * (2 + Config.cell_nodes)
        
        # First two nodes are cell inputs
        nodes[0] = x
        nodes[1] = x  # In a more complex setup, this could be a different input
        
        # Process each intermediate node
        for node_idx, (node_edges, node_ops) in enumerate(zip(self.genotype.edges, self.genotype.ops)):
            actual_node_idx = node_idx + 2  # Offset for input nodes
            
            node_inputs = []
            for edge_idx, (input_idx, op_name) in enumerate(zip(node_edges, node_ops)):
                # Apply operation to input
                processed = NASOperations.apply_operation(
                    nodes[input_idx], 
                    op_name, 
                    filters, 
                    f"{name_prefix}_n{actual_node_idx}_e{edge_idx}"
                )
                
                if processed is not None:  # Skip 'none' operations
                    node_inputs.append(processed)
            
            # Combine inputs if there are any
            if node_inputs:
                if len(node_inputs) == 1:
                    combined = node_inputs[0]
                else:
                    combined = Add()(node_inputs)
                
                # Apply activation and batch norm
                combined = BatchNormalization()(combined)
                combined = LeakyReLU(alpha=0.2)(combined)
                
                nodes[actual_node_idx] = combined
            else:
                # If no inputs, use skip connection from previous node
                nodes[actual_node_idx] = nodes[actual_node_idx-1]
        
        # Use the last node as cell output
        return nodes[-1]



In [7]:
# Standard GAN implementation
class StandardGAN:
    def __init__(self, X_train):
        self.X_train = X_train
        self.latent_dim = Config.latent_dim
        self.img_shape = (Config.img_width, Config.img_height, Config.channels)
        
        # Build and compile the discriminator
        self.discriminator = self._build_discriminator()
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=Adam(0.0002, 0.5),
            metrics=['accuracy']
        )
        
        # Build the generator
        self.generator = self._build_generator()
        
        # For the combined model, we will only train the generator
        self.discriminator.trainable = False
        
        # The generator takes noise as input and generates images
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)
        
        # The combined model (stacked generator and discriminator)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    
    def _build_generator(self):
        model = Sequential()
        
        # Foundation for 8x8 feature maps
        model.add(Dense(8*8*256, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 256)))
        
        # Upsampling to 16x16
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        
        # Upsampling to 32x32
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        
        # Upsampling to 64x64
        model.add(UpSampling2D())
        model.add(Conv2D(32, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        
        # Output layer with 3 channels (RGB)
        model.add(Conv2D(3, kernel_size=3, padding="same", activation="tanh"))
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    def _build_discriminator(self):
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(negative_slope=0.2))  # Changed from alpha to negative_slope
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(negative_slope=0.2))  # Changed from alpha to negative_slope
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(negative_slope=0.2))  # Changed from alpha to negative_slope
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(negative_slope=0.2))  # Changed from alpha to negative_slope
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        
        return model
    
    def train(self, epochs, batch_size=32, save_interval=5, save_prefix="standard"):
        # Save training history
        d_losses = []
        g_losses = []
        d_accs = []
        
        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # Select a random batch of images
            idx = np.random.randint(0, self.X_train.shape[0], batch_size // 2)
            imgs = self.X_train[idx]
            
            # Generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size // 2, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((batch_size // 2, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((batch_size // 2, 1)))
            
            # Check if d_loss is a list/tuple with two elements or just a single value
            if isinstance(d_loss_real, (list, tuple)) and len(d_loss_real) > 1:
                d_loss = [0.5 * (d_loss_real[0] + d_loss_fake[0]), 0.5 * (d_loss_real[1] + d_loss_fake[1])]
                d_loss_value = d_loss[0]
                d_acc_value = d_loss[1]
            else:
                d_loss = 0.5 * (d_loss_real + d_loss_fake)
                d_loss_value = d_loss
                d_acc_value = 0.5  # Default accuracy if not provided
            
            # ---------------------
            #  Train Generator
            # ---------------------
            
            # Train the generator to fool the discriminator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))
            
            # Save losses and accuracy for plotting
            d_losses.append(d_loss_value)
            d_accs.append(d_acc_value)
            g_losses.append(g_loss)
            
            # Print progress
            g_loss_value = g_loss[0] if isinstance(g_loss, (list, tuple)) else g_loss
            print(f"{epoch}/{epochs} [D loss: {d_loss_value:.4f}, acc.: {100*d_acc_value:.2f}%] [G loss: {g_loss_value:.4f}]")
            
            
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch, save_prefix)
        
        # Plot training history
        self.plot_history(d_losses, g_losses, d_accs, save_prefix)
        
        return d_losses, g_losses, d_accs
        
    def save_imgs(self, epoch, prefix, examples=10):
        """Save generated images."""
        noise = np.random.normal(0, 1, (examples, self.latent_dim))
        generated_images = self.generator.predict(noise)
        
        # Rescale images from [-1, 1] to [0, 1]
        generated_images = (generated_images + 1) / 2.0
        
        plt.figure(figsize=(10, 4))
        for i in range(examples):
            plt.subplot(2, 5, i+1)
            plt.imshow(generated_images[i])
            plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'{prefix}_gan_generated_image_epoch_{epoch}.png')
        plt.close()
        
        return generated_images
    
    def plot_history(self, d_losses, g_losses, d_accs, prefix):
        plt.figure(figsize=(15, 5))
        
        # Plot discriminator loss
        plt.subplot(1, 3, 1)
        plt.plot(d_losses)
        plt.title('Discriminator Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        
        # Plot generator loss
        plt.subplot(1, 3, 2)
        plt.plot(g_losses)
        plt.title('Generator Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        
        # Plot discriminator accuracy
        plt.subplot(1, 3, 3)
        plt.plot(d_accs)
        plt.title('Discriminator Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        
        plt.tight_layout()
        plt.savefig(f'{prefix}_gan_training_history.png')
        plt.close()
    
    def generate_augmented_images(self, num_images=5000):
        """Generate augmented images."""
        noise = np.random.normal(0, 1, (num_images, self.latent_dim))
        generated_images = self.generator.predict(noise)
        
        # Convert from [-1, 1] to [0, 1]
        generated_images = (generated_images + 1) / 2.0
        
        return generated_images



In [8]:
# NAS-GAN implementation using evolutionary search
class NASGAN:
    def __init__(self, X_train):
        self.X_train = X_train
        self.latent_dim = Config.latent_dim
        self.population = []
        self.fitness_history = []
        self.best_genotype = None
        self.best_fitness = -float('inf')
        self.img_shape = (Config.img_width, Config.img_height, Config.channels)
    
    def initialize_population(self):
        """Initialize random population of architectures."""
        self.population = [CellGenotype() for _ in range(Config.population_size)]
    
    def _build_discriminator(self):
        """Build a fixed discriminator architecture."""
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3, strides=2, 
                         input_shape=self.img_shape, 
                         padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        
        return model
    
    def evaluate_fitness(self, genotype, epochs=3):
        """Evaluate fitness of a single genotype."""
        # Build generator with this genotype
        nas_gen = NASGANGenerator(genotype)
        generator = nas_gen.build_model()
        
        # Build discriminator (fixed architecture)
        discriminator = self._build_discriminator()
        discriminator.compile(
            loss='binary_crossentropy',
            optimizer=Adam(0.0002, 0.5),
            metrics=['accuracy']
        )
        
        # Create GAN
        discriminator.trainable = False
        z = Input(shape=(self.latent_dim,))
        img = generator(z)
        valid = discriminator(img)
        combined = Model(z, valid)
        combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
        
        # Train for a few epochs to assess performance
        batch_size = Config.batch_size
        half_batch = batch_size // 2
        
        # Keep track of metrics
        d_losses = []
        g_losses = []
        
        for epoch in range(epochs):
            # Train discriminator
            idx = np.random.randint(0, self.X_train.shape[0], half_batch)
            real_imgs = self.X_train[idx]
            
            noise = np.random.normal(0, 1, (half_batch, self.latent_dim))
            gen_imgs = generator.predict(noise)
            
            d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
            
            d_losses.append(d_loss[0])
            g_losses.append(g_loss)
        
        # Use generator loss stability and discriminator accuracy as fitness
        # Lower generator loss is better, but we also want the discriminator to be accurate
        g_loss_stability = -np.std(g_losses[-5:]) if len(g_losses) >= 5 else -np.std(g_losses)
        d_accuracy = np.mean([d[1] for d in d_losses]) if isinstance(d_losses[0], (list, tuple)) else 0.5
        
        # A fitness function that balances generator quality and training stability
        fitness = g_loss_stability - abs(0.7 - d_accuracy) * 2
        
        # Clean up to prevent memory leaks
        tf.keras.backend.clear_session()
        
        return fitness
    
    def select_parent(self):
        """Select parent using tournament selection."""
        tournament = random.sample(list(enumerate(self.population)), Config.tournament_size)
        fitness_scores = []
        
        for idx, genotype in tournament:
            # Try to retrieve cached fitness if available
            if idx < len(self.fitness_history) and self.fitness_history[idx] is not None:
                fitness = self.fitness_history[idx]
            else:
                fitness = self.evaluate_fitness(genotype)
                # Cache the fitness
                if idx >= len(self.fitness_history):
                    self.fitness_history.extend([None] * (idx + 1 - len(self.fitness_history)))
                self.fitness_history[idx] = fitness
            
            fitness_scores.append((genotype, fitness))
        
        # Return the genotype with the best fitness
        return max(fitness_scores, key=lambda x: x[1])[0]
    
    def evolve(self):
        """Run the evolutionary search for optimal architectures."""
        print("Initializing population...")
        self.initialize_population()
        
        for generation in range(Config.generations):
            print(f"\nGeneration {generation+1}/{Config.generations}")
            
            # Evaluate fitness for all individuals in the population
            fitness_scores = []
            for i, genotype in enumerate(tqdm(self.population, desc="Evaluating fitness")):
                fitness = self.evaluate_fitness(genotype, epochs=Config.nas_epochs)
                fitness_scores.append(fitness)
                
                # Update best genotype if necessary
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_genotype = deepcopy(genotype)
            
            self.fitness_history = fitness_scores
            
            # Create next generation through selection, crossover, and mutation
            new_population = []
            
            # Elitism: keep the best individual
            best_idx = np.argmax(fitness_scores)
            new_population.append(deepcopy(self.population[best_idx]))
            
            # Generate rest of the population
            while len(new_population) < Config.population_size:
                # Select parents
                parent1 = self.select_parent()
                parent2 = self.select_parent()
                
                # Crossover
                if random.random() < 0.5:
                    child = parent1.crossover(parent2)
                else:
                    child = deepcopy(random.choice([parent1, parent2]))
                
                # Mutation
                if random.random() < Config.mutation_rate:
                    child = child.mutate()
                
                new_population.append(child)
            
            # Replace old population
            self.population = new_population
            
            # Print statistics
            avg_fitness = np.mean(fitness_scores)
            best_fitness = np.max(fitness_scores)
            print(f"Generation {generation+1} stats: Avg fitness = {avg_fitness:.4f}, Best fitness = {best_fitness:.4f}")
            print(f"Best genotype so far: {self.best_genotype}")
        
        print("\nEvolution complete. Best genotype:")
        print(self.best_genotype)
        return self.best_genotype
    
    def train_best_model(self, epochs=10, batch_size=32, save_interval=5):
        """Train the best discovered architecture for more epochs."""
        if self.best_genotype is None:
            raise ValueError("No best genotype found. Run evolve() first.")
        
        # Build generator with best genotype
        nas_gen = NASGANGenerator(self.best_genotype)
        generator = nas_gen.build_model()
        
        # Build discriminator
        discriminator = self._build_discriminator()
        discriminator.compile(
            loss='binary_crossentropy',
            optimizer=Adam(0.0002, 0.5),
            metrics=['accuracy']
        )
        
        # Create GAN
        discriminator.trainable = False
        z = Input(shape=(self.latent_dim,))
        img = generator(z)
        valid = discriminator(img)
        combined = Model(z, valid)
        combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
        
        # Training history
        d_losses = []
        g_losses = []
        d_accs = []
        
        half_batch = batch_size // 2
        
        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # Select a random batch of images
            idx = np.random.randint(0, self.X_train.shape[0], half_batch)
            imgs = self.X_train[idx]
            
            # Generate a batch of new images
            noise = np.random.normal(0, 1, (half_batch, self.latent_dim))
            gen_imgs = generator.predict(noise)
            
            # Train the discriminator
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # ---------------------
            #  Train Generator
            # ---------------------
            
            # Train the generator to fool the discriminator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
            
            # Save losses and accuracy for plotting
            d_losses.append(d_loss[0])
            d_accs.append(d_loss[1])
            g_losses.append(g_loss)
            
            # Print progress
            # Extract loss values
            d_loss_value = float(d_loss[0]) if isinstance(d_loss, (list, tuple, np.ndarray)) else float(d_loss)
            d_acc_value = float(d_loss[1]) if isinstance(d_loss, (list, tuple, np.ndarray)) and len(d_loss) > 1 else 0.0
            g_loss_value = float(g_loss[0]) if isinstance(g_loss, (list, tuple, np.ndarray)) else float(g_loss)

            
            # Print progress
            print(f"{epoch}/{epochs} [D loss: {d_loss_value:.4f}, acc.: {100*d_acc_value:.2f}%] [G loss: {g_loss_value:.4f}]")

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(generator, epoch, "nas")
        
        # Plot training history
        self.plot_history(d_losses, g_losses, d_accs, "nas")
        
        return generator, discriminator, (d_losses, g_losses, d_accs)
    
    def save_imgs(self, generator, epoch, prefix, examples=10):
        """Save generated images."""
        noise = np.random.normal(0, 1, (examples, self.latent_dim))
        generated_images = generator.predict(noise)
        
        # Rescale images from [-1, 1] to [0, 1]
        generated_images = (generated_images + 1) / 2.0
        
        plt.figure(figsize=(10, 4))
        for i in range(examples):
            plt.subplot(2, 5, i+1)
            plt.imshow(generated_images[i])
            plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'{prefix}_gan_generated_image_epoch_{epoch}.png')
        plt.close()
        
        return generated_images
    
    def plot_history(self, d_losses, g_losses, d_accs, prefix):
        plt.figure(figsize=(15, 5))
        
        # Plot discriminator loss
        plt.subplot(1, 3, 1)
        plt.plot(d_losses)
        plt.title('Discriminator Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        
        # Plot generator loss
        plt.subplot(1, 3, 2)
        plt.plot(g_losses)
        plt.title('Generator Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        
        # Plot discriminator accuracy
        plt.subplot(1, 3, 3)
        plt.plot(d_accs)
        plt.title('Discriminator Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        
        plt.tight_layout()
        plt.savefig(f'{prefix}_gan_training_history.png')
        plt.close()
    
    def generate_augmented_images(self, num_images=5000):
        """Generate augmented images using the best architecture."""
        if self.best_genotype is None:
            raise ValueError("No best genotype found. Run evolve() first.")
        
        nas_gen = NASGANGenerator(self.best_genotype)
        generator = nas_gen.build_model()
        
        noise = np.random.normal(0, 1, (num_images, self.latent_dim))
        generated_images = generator.predict(noise)
        
        # Convert from [-1, 1] to [0, 1]
        generated_images = (generated_images + 1) / 2.0
        
        return generated_images




In [9]:
def build_classifier(input_shape=(64, 64, 3), augmented=False):
    """Build a CNN classifier for malaria detection."""
    model = Sequential()
    
    # First convolutional block
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Second convolutional block
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Third convolutional block
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Classification block
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),  # Changed from lr to learning_rate
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model



In [10]:
# Evaluation function
def evaluate_classifier(model, X_test, y_test):
    """Evaluate classifier performance."""
    # Predict on test set
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Calculate metrics
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Uninfected', 'Parasite'])
    plt.yticks(tick_marks, ['Uninfected', 'Parasite'])
    
    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j],
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    return y_pred, cm


In [11]:
# Main execution function
def run_experiment():
    """Run the complete experiment."""
    # Load data
    print("Loading data...")
    train_gen, test_gen = create_data_generators()
    X_train, y_train = load_data_from_generator(train_gen)
    X_test, y_test = load_data_from_generator(test_gen)
    
    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")
    
    # 1. Train a standard CNN classifier (baseline)
    print("\n1. Training baseline classifier...")
    baseline_classifier = build_classifier()
    baseline_history = baseline_classifier.fit(
        X_train, y_train,
        epochs=15,
        batch_size=Config.batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    print("\nEvaluating baseline classifier...")
    baseline_preds, baseline_cm = evaluate_classifier(baseline_classifier, X_test, y_test)
    
    # 2. Train a standard GAN and use it for data augmentation
    print("\n2. Training standard GAN...")
    standard_gan = StandardGAN(X_train)
    standard_gan.train(epochs=Config.standard_gan_epochs, batch_size=Config.batch_size)
    
    # Generate augmented data
    print("Generating augmented images from standard GAN...")
    standard_gan_images = standard_gan.generate_augmented_images(num_images=len(X_train))
    
    # Create augmented dataset
    standard_augmented_X = np.concatenate([X_train, standard_gan_images*2-1], axis=0)  # Scale back to [-1, 1]
    standard_augmented_y = np.concatenate([y_train, np.zeros(len(standard_gan_images))], axis=0)  # Label generated images as uninfected
    
    # Train classifier with standard GAN augmentation
    print("Training classifier with standard GAN augmentation...")
    standard_gan_classifier = build_classifier()
    standard_gan_history = standard_gan_classifier.fit(
        standard_augmented_X, standard_augmented_y,
        epochs=15,
        batch_size=Config.batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    print("\nEvaluating standard GAN-augmented classifier...")
    standard_gan_preds, standard_gan_cm = evaluate_classifier(standard_gan_classifier, X_test, y_test)
    
    # 3. Run NAS to find optimal GAN architecture
    print("\n3. Running Neural Architecture Search for GAN...")
    nas_gan = NASGAN(X_train)
    best_genotype = nas_gan.evolve()
    
    # Train the best architecture for more epochs
    print("Training best NAS-discovered GAN architecture...")
    best_generator, _, _ = nas_gan.train_best_model(epochs=Config.final_epochs)
    
    # Generate augmented data with the NAS-GAN
    print("Generating augmented images from NAS-GAN...")
    nas_gan_images = nas_gan.generate_augmented_images(num_images=len(X_train))
    
    # Create augmented dataset
    nas_augmented_X = np.concatenate([X_train, nas_gan_images*2-1], axis=0)  # Scale back to [-1, 1]
    nas_augmented_y = np.concatenate([y_train, np.zeros(len(nas_gan_images))], axis=0)  # Label generated images as uninfected
    
    # Train classifier with NAS-GAN augmentation
    print("Training classifier with NAS-GAN augmentation...")
    nas_gan_classifier = build_classifier()
    nas_gan_history = nas_gan_classifier.fit(
        nas_augmented_X, nas_augmented_y,
        epochs=15,
        batch_size=Config.batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    print("\nEvaluating NAS-GAN-augmented classifier...")
    nas_gan_preds, nas_gan_cm = evaluate_classifier(nas_gan_classifier, X_test, y_test)
    
    # 4. Compare results
    print("\n4. Comparing results of all models...")
    
    # Plot training histories
    plt.figure(figsize=(12, 5))
    
    # Accuracy comparison
    plt.subplot(1, 2, 1)
    plt.plot(baseline_history.history['val_accuracy'], label='Baseline')
    plt.plot(standard_gan_history.history['val_accuracy'], label='Standard GAN')
    plt.plot(nas_gan_history.history['val_accuracy'], label='NAS-GAN')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Loss comparison
    plt.subplot(1, 2, 2)
    plt.plot(baseline_history.history['val_loss'], label='Baseline')
    plt.plot(standard_gan_history.history['val_loss'], label='Standard GAN')
    plt.plot(nas_gan_history.history['val_loss'], label='NAS-GAN')
    plt.title('Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('model_comparison.png')
    plt.close()
    
    print("\nExperiment complete! Results saved to disk.")
    
    return {
        'baseline_classifier': baseline_classifier,
        'standard_gan_classifier': standard_gan_classifier,
        'nas_gan_classifier': nas_gan_classifier,
        'best_genotype': best_genotype
    }



In [12]:
# If this script is run directly, execute the experiment
if __name__ == "__main__":
    print("Starting NAS-GAN experiment for malaria detection...")
    start_time = time.time()
    results = run_experiment()
    end_time = time.time()
    
    print(f"\nTotal experiment time: {(end_time - start_time)/60:.2f} minutes")
    print("Best GAN architecture discovered:")
    print(results['best_genotype'])

Starting NAS-GAN experiment for malaria detection...
Loading data...
Found 416 images belonging to 2 classes.
Found 134 images belonging to 2 classes.
Training data shape: (416, 64, 64, 3)
Test data shape: (134, 64, 64, 3)

1. Training baseline classifier...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745548446.508946      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745548446.509619      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/15


I0000 00:00:1745548452.121529     109 service.cc:148] XLA service 0x7bbb740091a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745548452.122352     109 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745548452.122374     109 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745548452.597786     109 cuda_dnn.cc:529] Loaded cuDNN version 90300


 7/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5094 - loss: 0.8792  

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


Evaluating fitness:  90%|█████████ | 9/10 [04:14<00:28, 28.23s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Evaluating fitness: 100%|██████████| 10/10 [04:42<00:00, 28.24s/it]


Generation 5 stats: Avg fitness = -0.5151, Best fitness = -0.4293
Best genotype so far: Node 2: (0->2, dil_conv5x5) (1->2, skip_connect) 
Node 3: (2->3, conv3x3) 
Node 4: (0->4, conv3x3) 
Node 5: (2->5, conv7x7) (1->5, skip_connect) 

Evolution complete. Best genotype:
Node 2: (0->2, dil_conv5x5) (1->2, skip_connect) 
Node 3: (2->3, conv3x3) 
Node 4: (0->4, conv3x3) 
Node 5: (2->5, conv7x7) (1->5, skip_connect) 
Training best NAS-discovered GAN architecture...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0/10 [D loss: 0.6746, acc.: 60.94%] [G loss: 0.6849]
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/10 [D loss: 0.6844, acc.: 49.22%] [G loss: 0.6890]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/10 [D loss: 0.6894, acc.: 45.83%] [G loss: 0.6933]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
3/10 [D loss: 0.6935, acc.: 45.20%] [G loss: 0.6978]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4/10 [D loss: 0.6989, acc.: 44.20%] [G loss: 0.7030]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
5/10 [D 